#### Họ và tên: Trần Trung Hiếu
#### MSSV: 1712442

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## Giới thiệu dữ liệu

Dữ liệu <i>FIFA 19 complete player dataset</i> được lấy từ Kaggle tại [đây](https://www.kaggle.com/karangadiya/fifa19).

Đây là dữ liệu chứa thông tin về các cầu thủ của FIFA vào năm 2019 của [SoFIFA](https://sofifa.com/). Dữ liệu được tác giả thu thập bằng thư viện BeautifulSoup và requests trong Python.

Theo thông tin về license mà tác giả đưa ra trên Kaggle, CC BY-NC-SA 4.0 cho phép người sử dụng chia sẻ và tùy chỉnh dữ liệu.


In [37]:
# Đọc file csv
football_df = pd.read_csv('data.csv', index_col=[0])

## Khám phá dữ liệu

In [38]:
football_df.head()

,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,https://cdn.sofifa.org/teams/2/light/10.png,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


In [39]:
print("Number of rows: %d\nNumber of columns: %d" % (football_df.shape))

Number of rows: 18207
Number of columns: 88


In [99]:
# Liệt kê các cột ứng với kiểu dữ liệu và số lượng dòng khác rỗng
football_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18159 entries, 0 to 18206
Data columns (total 47 columns):
ID                 18159 non-null int64
Name               18159 non-null object
Age                18159 non-null int64
Nationality        18159 non-null object
Club               18159 non-null object
Value              18159 non-null float64
Wage               18159 non-null float64
Overall            18159 non-null int64
Potential          18159 non-null int64
Preferred Foot     18159 non-null object
Position           18147 non-null object
Height             18159 non-null object
Weight             18159 non-null object
Crossing           18159 non-null float64
Finishing          18159 non-null float64
HeadingAccuracy    18159 non-null float64
ShortPassing       18159 non-null float64
Volleys            18159 non-null float64
Dribbling          18159 non-null float64
Curve              18159 non-null float64
FKAccuracy         18159 non-null float64
LongPassing        18159

In [58]:
# Đếm các giá trị null
football_df.isnull().sum()

ID                  0
Name                0
Age                 0
Nationality         0
Club                0
Value               0
Wage                0
Overall             0
Potential           0
Preferred Foot      0
Position           12
Height              0
Weight              0
Crossing            0
Finishing           0
HeadingAccuracy     0
ShortPassing        0
Volleys             0
Dribbling           0
Curve               0
FKAccuracy          0
LongPassing         0
BallControl         0
Acceleration        0
SprintSpeed         0
Agility             0
Reactions           0
Balance             0
ShotPower           0
Jumping             0
Stamina             0
Strength            0
LongShots           0
Aggression          0
Interceptions       0
Positioning         0
Vision              0
Penalties           0
Composure           0
Marking             0
StandingTackle      0
SlidingTackle       0
GKKicking           0
GKPositioning       0
GKReflexes          0
GKDiving  

## Tiền xử lí dữ liệu

#### Loại bỏ các cột không cần thiết

Đầu tiên cần loại bỏ bớt các cột không cần thiết trong bài phân tích này dựa trên những cột được liệt kê phía trên.

Như quan sát, những cột như Joined, Loaned From, Contract Valid Until, Release Clause và cùng với LS, ST, RS, LW, LF, CF, RF, RW, LAM, CAM, RAM, LM, LCM, CM, RCM, RM, LWB, LDM, CDM, RDM, RWB, LB, LCB, CB, RCB, RB là những cột có giá trị thiếu rất nhiều, và những cột này cũng không cần thiết trong quá trình phân tích cho nên ta sẽ loại bỏ nó và đồng thời cũng loại bỏ một số cột không cần thiết khác.

Dưới đây là mô tả về các cột được sử dụng trong phân tích của dữ liệu FIFA 19 complete player:

+ ID: chỉ số định danh trên FIFA

+ Name: tên cầu thủ.

+ Age: tuổi cầu thủ.

+ Nationality: quốc tịch cầu thủ.

+ Club: câu lạc bộ đang thi đấu.

+ Value: giá trị chuyển nhượng.

+ Wage: lương của cầu thủ.

+ Overall: đánh giá tổng quan.

+ Potential: tiềm năng.

+ Preferred Foot: chân thuận.

+ Position: vị trí trong sân.

+ Height/Weight: chiều cao và cân nặng của cầu thủ.

+ Crossing, Finishing, HeadingAccuracy, ShortPassing, Volleys: các chỉ số về tấn công.

+ Dribbling, Curve, FKAccuracy, LongPassing, BallControl: các chỉ số về kĩ năng.

+ Acceleration, SprintSpeed, Agility, Reactions, Balance: các chỉ số về di chuyển.

+ ShotPower, Jumping, Stamina, Strength, LongShots: các chỉ số về sức mạnh.

+ Aggression, Interceptions, Positioning, Vision, Penalties, Composure: các chỉ số về tinh thần.

+ Marking, StandingTackle, SlidingTackle : các chỉ số về phòng thủ.

+ GKKicking, GKPositioning, GKReflexes, GKDiving, GKHandling: các chỉ số về giữ lưới thủ môn.

In [43]:
football_df = football_df[[
    'ID', 'Name', 'Age', 'Nationality', 'Club', 'Value', 'Wage', 'Overall', 'Potential', 
    'Preferred Foot', 'Position', 'Height', 'Weight', 
    'Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys',
    'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing', 'BallControl',
    'Acceleration', 'SprintSpeed', 'Agility', 'Reactions', 'Balance', 
    'ShotPower', 'Jumping', 'Stamina', 'Strength', 'LongShots',
    'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
    'Marking', 'StandingTackle', 'SlidingTackle',
    'GKKicking', 'GKPositioning', 'GKReflexes', 'GKDiving', 'GKHandling'
]]

,ID,Name,Age,Nationality,Club,Value,Wage,Overall,Potential,Preferred Foot,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKKicking,GKPositioning,GKReflexes,GKDiving,GKHandling
0,158023,L. Messi,31,Argentina,FC Barcelona,€110.5M,€565K,94,94,Left,...,75.0,96.0,33.0,28.0,26.0,15.0,14.0,8.0,6.0,11.0
1,20801,Cristiano Ronaldo,33,Portugal,Juventus,€77M,€405K,94,94,Right,...,85.0,95.0,28.0,31.0,23.0,15.0,14.0,11.0,7.0,11.0
2,190871,Neymar Jr,26,Brazil,Paris Saint-Germain,€118.5M,€290K,92,93,Right,...,81.0,94.0,27.0,24.0,33.0,15.0,15.0,11.0,9.0,9.0
3,193080,De Gea,27,Spain,Manchester United,€72M,€260K,91,93,Right,...,40.0,68.0,15.0,21.0,13.0,87.0,88.0,94.0,90.0,85.0
4,192985,K. De Bruyne,27,Belgium,Manchester City,€102M,€355K,91,92,Right,...,79.0,88.0,68.0,58.0,51.0,5.0,10.0,13.0,15.0,13.0


In [67]:
print("Number of rows: %d\nNumber of columns: %d" % (football_df.shape))

Number of rows: 18159
Number of columns: 47


#### Xử lí các giá trị bị thiếu

Đối với những dòng có Club là NaN, có thể là những cầu thủ này chưa có câu lạc bộ cho nên ta sẽ điền 'No Club' vào.

In [45]:
football_df['Club'].fillna('No Club', inplace=True)

Liệt kê các dòng còn giá trị thiếu để xem xét xử lí.

In [57]:
football_df[football_df.isnull().any(axis=1)]

,ID,Name,Age,Nationality,Club,Value,Wage,Overall,Potential,Preferred Foot,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKKicking,GKPositioning,GKReflexes,GKDiving,GKHandling
5018,153160,R. Raldes,37,Bolivia,No Club,€0,€0,70,70,Right,...,77.0,64.0,79.0,70.0,70.0,11.0,12.0,13.0,16.0,8.0
6736,175393,J. Arce,33,Bolivia,No Club,€0,€0,68,68,Left,...,69.0,67.0,12.0,34.0,33.0,15.0,16.0,8.0,16.0,13.0
7922,195905,L. Gutiérrez,33,Bolivia,No Club,€0,€0,67,67,Right,...,42.0,54.0,72.0,71.0,64.0,8.0,12.0,12.0,11.0,7.0
9905,226044,R. Vargas,23,Bolivia,No Club,€0,€0,66,69,Right,...,67.0,64.0,19.0,24.0,23.0,11.0,7.0,9.0,13.0,11.0
10628,216751,D. Bejarano,26,Bolivia,No Club,€0,€0,65,66,Right,...,39.0,57.0,68.0,69.0,68.0,11.0,14.0,12.0,14.0,12.0
16450,193911,S. Paul,31,India,No Club,€0,€0,57,57,Right,...,16.0,52.0,12.0,15.0,12.0,62.0,57.0,60.0,53.0,48.0
16539,245167,L. Lalruatthara,23,India,No Club,€0,€0,57,63,Left,...,38.0,57.0,60.0,61.0,57.0,11.0,6.0,12.0,15.0,12.0
16793,228192,E. Lyngdoh,31,India,No Club,€0,€0,56,56,Right,...,59.0,63.0,43.0,48.0,32.0,9.0,15.0,9.0,7.0,8.0
17129,228198,J. Singh,26,India,No Club,€0,€0,55,58,Right,...,65.0,42.0,26.0,18.0,13.0,8.0,10.0,16.0,15.0,14.0
17339,233526,S. Passi,23,India,No Club,€0,€0,54,63,Right,...,58.0,45.0,14.0,23.0,21.0,13.0,9.0,10.0,8.0,6.0


Theo quan sát, một số các cầu thủ có nhiều cột chỉ số bị thiếu, cho nên ta sẽ thực hiện loại bỏ các cầu thủ đó và chỉ giữ lại các cầu thủ có ít nhất 11 cột không bị thiếu giá trị, đây là những cột chứa thông tin cá nhân của cầu thủ.

In [56]:
football_df.dropna(thresh=11, inplace=True)

In [71]:
# Đếm các giá trị null
football_df.isnull().sum()

ID                  0
Name                0
Age                 0
Nationality         0
Club                0
Value               0
Wage                0
Overall             0
Potential           0
Preferred Foot      0
Position           12
Height              0
Weight              0
Crossing            0
Finishing           0
HeadingAccuracy     0
ShortPassing        0
Volleys             0
Dribbling           0
Curve               0
FKAccuracy          0
LongPassing         0
BallControl         0
Acceleration        0
SprintSpeed         0
Agility             0
Reactions           0
Balance             0
ShotPower           0
Jumping             0
Stamina             0
Strength            0
LongShots           0
Aggression          0
Interceptions       0
Positioning         0
Vision              0
Penalties           0
Composure           0
Marking             0
StandingTackle      0
SlidingTackle       0
GKKicking           0
GKPositioning       0
GKReflexes          0
GKDiving  

#### Chuyển đổi Value và Wage từ object về dạng numeric

In [61]:
football_df.Value.unique(), football_df.Wage.unique()

(array(['€110.5M', '€77M', '€118.5M', '€72M', '€102M', '€93M', '€67M',
        '€80M', '€51M', '€68M', '€76.5M', '€44M', '€60M', '€63M', '€89M',
        '€83.5M', '€78M', '€58M', '€53.5M', '€51.5M', '€38M', '€64.5M',
        '€27M', '€81M', '€69.5M', '€59.5M', '€62M', '€73.5M', '€59M',
        '€46M', '€43M', '€36M', '€57M', '€24M', '€30M', '€4M', '€64M',
        '€30.5M', '€62.5M', '€52M', '€45M', '€34M', '€46.5M', '€61M',
        '€41.5M', '€44.5M', '€56.5M', '€53M', '€50M', '€55M', '€36.5M',
        '€45.5M', '€43.5M', '€35M', '€39M', '€18M', '€21.5M', '€50.5M',
        '€54M', '€40.5M', '€37.5M', '€28.5M', '€37M', '€32M', '€26M',
        '€33M', '€38.5M', '€35.5M', '€9M', '€15.5M', '€22M', '€14M',
        '€42.5M', '€31.5M', '€42M', '€25M', '€29.5M', '€31M', '€24.5M',
        '€27.5M', '€29M', '€16.5M', '€23M', '€19M', '€4.2M', '€40M',
        '€41M', '€28M', '€22.5M', '€34.5M', '€32.5M', '€20M', '€26.5M',
        '€25.5M', '€21M', '€13M', '€17.5M', '€11.5M', '€8M', '€6M',
        

Các giá trị của Value và Wage là dạng chuỗi với tiền tố là kí hiệu tiền Euro, hậu tố là số lượng chữ số 0 như M = 6 số, K = 3 số

In [63]:
def convert_value(value):
    value = value[1:]
    if value.endswith('K'):
        return float(value[:-1]) * 1000
    elif value.endswith('M'):
        return float(value[:-1]) * 1000000
    else:
        return float(value)
    
football_df['Value'] = football_df.Value.apply(convert_value)
football_df['Wage'] = football_df.Wage.apply(convert_value)

In [68]:
football_df.sample(5)

,ID,Name,Age,Nationality,Club,Value,Wage,Overall,Potential,Preferred Foot,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKKicking,GKPositioning,GKReflexes,GKDiving,GKHandling
5880,211270,E. Saveljich,27,Montenegro,UD Almería,1100000.0,10000.0,69,72,Right,...,43.0,62.0,70.0,72.0,62.0,10.0,14.0,15.0,12.0,12.0
2035,230381,Kaimo Lima,34,Brazil,Fluminense,2000000.0,24000.0,75,75,Left,...,66.0,76.0,74.0,76.0,70.0,9.0,10.0,12.0,8.0,8.0
13795,190934,P. Uscanga,27,Mexico,Querétaro,350000.0,2000.0,62,63,Right,...,45.0,62.0,54.0,63.0,61.0,12.0,6.0,8.0,8.0,6.0
5710,45842,J. Olsson,35,Sweden,Djurgårdens IF,250000.0,2000.0,69,69,Left,...,51.0,55.0,64.0,68.0,66.0,16.0,14.0,11.0,9.0,12.0
8065,228699,D. Wouters,21,Belgium,KRC Genk,1000000.0,4000.0,67,76,Left,...,25.0,66.0,61.0,68.0,66.0,15.0,8.0,15.0,10.0,10.0


In [98]:
# Mô tả các giữ liệu số
football_df.describe()

,ID,Age,Value,Wage,Overall,Potential,Crossing,Finishing,HeadingAccuracy,ShortPassing,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKKicking,GKPositioning,GKReflexes,GKDiving,GKHandling
count,18159.000000,18159.000000,1.815900e+04,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,...,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000
mean,214279.590286,25.122529,2.416131e+06,9752.574481,66.249904,71.319126,49.734181,45.550911,52.298144,58.686712,...,48.548598,58.648274,47.281623,47.697836,45.661435,16.232061,16.388898,16.710887,16.616223,16.391596
std,29968.461880,4.670568,5.601319e+06,22024.397597,6.914613,6.134420,18.364524,19.525820,17.379909,14.699495,...,15.704053,11.436133,19.904397,21.664004,21.289135,16.502864,17.034669,17.955119,17.695349,16.906900
min,16.000000,16.000000,0.000000e+00,0.000000,46.000000,48.000000,5.000000,2.000000,4.000000,7.000000,...,5.000000,3.000000,3.000000,2.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,200300.000000,21.000000,3.000000e+05,1000.000000,62.000000,67.000000,38.000000,30.000000,44.000000,54.000000,...,39.000000,51.000000,30.000000,27.000000,24.000000,8.000000,8.000000,8.000000,8.000000,8.000000
50%,221743.000000,25.000000,6.750000e+05,3000.000000,66.000000,71.000000,54.000000,49.000000,56.000000,62.000000,...,49.000000,60.000000,53.000000,55.000000,52.000000,11.000000,11.000000,11.000000,11.000000,11.000000
75%,236508.500000,28.000000,2.000000e+06,9000.000000,71.000000,75.000000,64.000000,62.000000,64.000000,68.000000,...,60.000000,67.000000,64.000000,66.000000,64.000000,14.000000,14.000000,14.000000,14.000000,14.000000
max,246620.000000,45.000000,1.185000e+08,565000.000000,94.000000,95.000000,93.000000,95.000000,94.000000,93.000000,...,92.000000,96.000000,94.000000,93.000000,91.000000,91.000000,90.000000,94.000000,90.000000,92.000000


## Thống kê dữ liệu

#### 1. Số lượng các cầu thủ thi đấu ở các vị trí trong sân

In [123]:
print("Tổng số vị trí thi đấu:", football_df.Position.unique().size)
postion_count_df = football_df.Position.value_counts().to_frame().reset_index()
postion_count_df.columns = ['Position', 'Count']
postion_count_df.head(10)

Tổng số vị trí thi đấu: 28


,Position,Count
0,ST,2152
1,GK,2025
2,CB,1778
3,CM,1394
4,LB,1322
5,RB,1291
6,RM,1124
7,LM,1095
8,CAM,958
9,CDM,948


#### 2. Số lượng cầu thủ của các câu lạc bộ

In [118]:
print("Tổng số câu lạc bộ:", football_df.Club.unique().size - 1)
club_count_df = football_df[football_df.Club != 'No Club'].Club.value_counts().to_frame().reset_index()
club_count_df.columns = ['Club', 'Count']
club_count_df.head(10)

Tổng số câu lạc bộ: 651


,Club,Count
0,Manchester United,33
1,Southampton,33
2,Arsenal,33
3,AS Monaco,33
4,Frosinone,33
5,TSG 1899 Hoffenheim,33
6,Burnley,33
7,Atlético Madrid,33
8,Cardiff City,33
9,Empoli,33


#### 3. Số lượng cầu thủ theo từng quốc gia

In [122]:
print("Tổng số quốc gia:", football_df.Nationality.unique().size)
nationality_count_df = football_df.Nationality.value_counts().to_frame().reset_index()
nationality_count_df.columns = ['Nationality', 'Count']
nationality_count_df.head(10)

Tổng số quốc gia: 164


,Nationality,Count
0,England,1657
1,Germany,1195
2,Spain,1071
3,Argentina,936
4,France,911
5,Brazil,825
6,Italy,699
7,Colombia,616
8,Japan,478
9,Netherlands,452


## Phân tích dữ liệu

1. Cầu thủ trẻ có giá trị cao nhất?

2. Danh sách các câu lạc bộ có giá trị cao nhất

3. Thống kê các cầu thủ xuất sắc nhất theo vị trí

4. So sánh cầu thủ trẻ và cầu thủ già về giá trị và tiềm năng, liệu cầu thủ trẻ có lương nhiều hơn cầu thủ già

5. Biểu đồ thống kê cầu thủ từ các châu lục.

6. So sánh giữa vị trí thi đấu với tiền lương hoặc giá trị chuyển nhượng

7. Xây dựng mô hình dự đoán vị trí cho 12 cầu thủ còn thiếu
